# Hallucination Evaluation

In [1]:
# Install necessary libraries
!pip install deepeval
!pip install PyMuPDF
!pip install sentence-transformers
!pip install numpy==1.19.5 sentence-transformers 
import pandas as pd
import os
import time
from tqdm import tqdm
from deepeval import evaluate
from deepeval.metrics import HallucinationMetric
from deepeval.test_case import LLMTestCase
import logging
import warnings
import deepeval
import nest_asyncio

  Using cached grpcio-1.63.2-cp312-cp312-macosx_10_9_universal2.whl.metadata (3.2 kB)
  Using cached protobuf-4.25.5-cp37-abi3-macosx_10_9_universal2.whl.metadata (541 bytes)
Using cached grpcio-1.63.2-cp312-cp312-macosx_10_9_universal2.whl (10.1 MB)
Using cached protobuf-4.25.5-cp37-abi3-macosx_10_9_universal2.whl (394 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.0
    Uninstalling protobuf-5.29.0:
      Successfully uninstalled protobuf-5.29.0
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.67.1
    Uninstalling grpcio-1.67.1:
      Successfully uninstalled grpcio-1.67.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.68.0 requires grpcio>=1.68.0, but you have grpcio 1.63.2 which is incompatible.
grpcio-status 1.68.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.5 

/Users/rishikasrinivas/.local/lib/python3.12/site-packages/deepeval/__init__.py:49: UserWarning: You are using deepeval version 1.4.5, however version 2.0.1 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


# Deep Eval

In [2]:
import fitz,os
import numpy as np
import deepeval
# Load a pre-trained model'
OPENAI_API_KEY=""
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
import nltk
from nltk.stem import PorterStemmer
nltk.download("punkt")

# Initialize Python porter stemmer
ps = PorterStemmer()
def lemmatize(sent):
    return [ps.stem(word) for word in sent.split()]
#reading the pdf and hand annotation files
def read_pdf(pdf_file):
    start=False
    sentences=[]
    start_idx=0
    with fitz.open(pdf_file) as pdf_file:
        for page_index, page in enumerate(pdf_file):
            text = page.get_text("text").lower()
            text=text.split(". ")
            for sub in text:
                if 'abstract' in sub or 'intro' in sub:
                    start=True
                    if 'abstract' in sub:
                        sub.index('abstract')
                    else:
                        sub.index('intro')
                        
                if start:
                    sentences.append(sub)
                
    return sentences
def read_files(root_dir, hand):
    
    lines=[]
    for files in os.listdir(root_dir):
        if files[-4:] != '.pdf':
            continue
        sentences = read_pdf(f"{root_dir}/{files}")
        lines.extend(sentences)

    # read in hand annotations
    for p in hand.iterrows():
        rel = p[1]['rel']
        subj = p[1]['subj']
        obj = p[1]['obj']
        out=f"{subj} {rel} {obj}" 
        lines.append(out)


    return lines

    return lines
hand = pd.read_csv("../Results/ground_truth.csv")
sentences = read_files("../Docs",hand)
#computing cosine similarity
def vec(sentences):
    # Encode sentences
    embeddings = model.encode([sentences[0], sentences[1]])
    
    # Compute cosine similarity
    similarity = util.cos_sim(embeddings[0], embeddings[1])
    return similarity.item() # Value close to 1 indicates high similarity
    
#finding if the target string (relation triplet) is in the src (pdf + hand annotation)
def find(target, src):
    found=False
    matching_sentences=[]
    
    for idx,sentence in enumerate(src):
        subj, obj = f" {target.split()[0]} ", f" {target.split()[1]} "
        if subj in sentence and obj in sentence:
            idx = find_substring_index(sentences, subj)
            matching_sentence=sentence[idx:]
            matching_sentences.append(matching_sentence)
            found=True
            break 
            
        #pred=" ".join(lemmatize(target))
        #test=" ".join(lemmatize(sentence))
        #cos = vec([pred,test])
        
        elif subj in sentence:# or cos > 0.68:
            found=True
            st_idx=idx
            idx = find_substring_index(sentences, subj)
            matching_sentence=sentence[idx:]
            matching_sentences.append(matching_sentence)
            
    print(len(matching_sentences))
    return found, matching_sentences

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rishikasrinivas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
hall, files, pred_files

([0.2590909090909091],
 ['../Results/NewRels_Skip3_increments.csv',
  '../Results/Temperature1_WithoutExamples.csv'],
 ['Temperature0point2.csv',
  'ground_truth.csv',
  'eval_results',
  'Temperature1_WithoutExamples.csv',
  'Temperature1_WithoutExamples_cleaned.csv',
  'NewRels_Skip2_cummulative.csv',
  '.ipynb_checkpoints',
  'NewRels_Skip3_cummulative.csv',
  'NewRels_Skip2_increments.csv',
  'NewRels_Skip4_increments.csv',
  'Temperature1_WithExamples.csv',
  'NewRels_Skip3_increments.csv'])

In [12]:
import deepeval
from deepeval import evaluate
from deepeval.metrics import HallucinationMetric
from deepeval.test_case import LLMTestCase
import pandas as pd
pred_files = os.listdir("../Results")
hand = pd.read_csv("../Results/ground_truth.csv")
sentences = read_files("../Docs",hand)
def hallucination_test(actual_output, context, extractions):
    sub, obj = extractions[0], extractions[1]
    metric = HallucinationMetric(threshold=0.5)
    scores=[]
    for c in context:
        test_case = LLMTestCase(
            input=f"What is the relationship between {sub} and {obj}",
            actual_output={actual_output},
            context=[c]
        )
        
        metric.measure(test_case)
        scores.append(metric.score)
    test_case = LLMTestCase(
        input=f"What is the relationship between {sub} and {obj}",
        actual_output={actual_output},
        context=context
    )
    metric.measure(test_case)
    scores.append(metric.score)
    metric.score = min(scores)
    
    return metric.score

import threading

import multiprocessing 
def find_substring_index(lst, substring):
    for i, s in enumerate(lst):
        if substring in s:  # Check if the substring exists in the string
            return i
    return -1 
def hallucination(sentences, pred_file):
    predictions = pd.read_csv(pred_file)
    total_halluncination_score = 0
    for i,p in enumerate(predictions.iterrows()):
        print(i)
    
        ref = p[1]['ref']
        rel = p[1]['rel']
        subj = p[1]['subj']
        obj = p[1]['obj']
        out=f"{subj} {rel} {obj}"
        
        found, matching_sentences = find(out.lower(), sentences)
        print(len(matching_sentences))
        if found:
            halluncination_scores=[]
            for match in matching_sentences:
                idx = find_substring_index(sentences, match)
                sentence_window = [sentences[i] for i in range(idx-1, idx+1)]
                halluncination_score = hallucination_test(out,context=sentence_window, extractions = [subj, obj])
                halluncination_scores.append(halluncination_score)
                
                
            total_halluncination_score += min(halluncination_scores)
        else:
            print(f"Could not find a match for {out}")
            total_halluncination_score += 1
            continue
        
        
       
        
    print(len(predictions), total_halluncination_score)
    print(f"Average hallucination score for predictions_file: {pred_file} is {total_halluncination_score/len(predictions)}")
    return total_halluncination_score/len(predictions)
import logging
httpx_logger = logging.getLogger("httpx")
httpx_logger.setLevel(logging.WARNING)
if __name__ == "__main__":
    hall=[]
    files=[]
    for pred_file in ['Temperature0point2.csv']:
        print(f"============={pred_file}=================")
        if any(substring in pred_file for substring in [ 'cumm', '.ipynb', 'ground', 'Clean', 'result']):
            continue
        pred_file=f"../Results/{pred_file}"
        files.append(pred_file)
        hall.append(hallucination(sentences, pred_file))
        
      
       

Output()

=============Temperature0point2.csv=================
0
0
0
Could not find a match for Problem-solve therapy synonym PST
1
1
1


Output()

Output()

Output()

...

Output()

108
1
1


Output()

Output()

109 27.5
Average hallucination score for predictions_file: ../Results/Temperature0point2.csv is 0.25229357798165136


In [ ]:
Average hallucination score for predictions_file: ../Results/Temperature0point2.csv is 0.5137614678899083
Hallucination score for ./Results/Temperature1_WithoutExamples.csv is 0.1810344827586207
Average hallucination score for predictions_file: ../Results/NewRels_Skip2_increments.csv is 0.6424418604651163

Average hallucination score for predictions_file: ../Results/NewRels_Skip4_increments.csv is 0.6369047619047619
Average hallucination score for predictions_file: ../Results/NewRels_Skip3_increments.csv is 0.2590909090909091

In [46]:
!pip install --upgrade deepeval openai langchain httpx


  Using cached langchain_core-0.3.21-py3-none-any.whl.metadata (6.3 kB)
  Using cached numpy-1.26.4-cp312-cp312-macosx_11_0_arm64.whl.metadata (61 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 12.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.3 MB/s eta 0:00:00
Using cached langchain_core-0.3.21-py3-none-any.whl (409 kB)
Using cached numpy-1.26.4-cp312-cp312-macosx_11_0_arm64.whl (13.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.3
    Uninstalling numpy-2.1.3:
      Successfully uninstalled numpy-2.1.3
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.63.2
    Uninstalling grpcio-1.63.2:
      Successfully uninstalled grpcio-1.63.2
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.13
    Uninstalling langchain-core-0.3.13:
      Successfully uninstalled langchain-core-0.3.13
  Attempting uninstall: langchain
    Found existing installation

In [8]:
69.5/189

0.36772486772486773